In [1]:
pip install psycopg2

  Using cached psycopg2-2.9.9.tar.gz (384 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [38 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-39
      creating build/lib.linux-x86_64-cpython-39/psycopg2
      copying lib/extras.py -> build/lib.linux-x86_64-cpython-39/psycopg2
      copying lib/extensions.py -> build/lib.linux-x86_64-cpython-39/psycopg2
      copying lib/_range.py -> build/lib.linux-x86_64-cpython-39/psycopg2
      copying lib/pool.py -> build/lib.linux-x86_64-cpython-39/psycopg2
      copying lib/_ipaddress.py -> build/lib.linux-x86_64-cpython-39/psycopg2
      copying lib/errors.py -> build/lib.linux-x86_64-cpython-39/psycopg2
      copying lib/__init__.py -> build/lib.linux-x86_64-cpython-39/psycopg2
      copying lib/tz.py -> build/lib.linux-x86_

In [2]:
pip install configparser


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import pandas as pd
import psycopg2
from psycopg2 import sql
import configparser

In [5]:
config = configparser.ConfigParser()
config.read('db_config.ini')

PGEND_POINT = config['database']['PGEND_POINT']
PGUSER_NAME = config['database']['PGUSER_NAME']
PGPASSWORD = config['database']['PGPASSWORD']
PGDATABASE_NAME = config['database']['PGDATABASE_NAME']

# Setting up the Connection

In [6]:
def connect():

    # Set up a connection to the postgres server.
    conn_string = "host="+ PGEND_POINT +" port="+ "5432" +" dbname="+ PGDATABASE_NAME +" user=" + PGUSER_NAME \
                  +" password="+ PGPASSWORD

    conn = psycopg2.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()

    return conn, cursor


In [7]:
conn, cursor = connect()

Connected!


# Reading the RUL Table

In [8]:
rul_query = sql.SQL("SELECT * FROM rultable;")

# Execute the query
cur = conn.cursor()
cur.execute(rul_query)

# Fetch all rows
rul_data = cur.fetchall()

# Fetch column names from the cursor description
column_names = [desc[0] for desc in cur.description]

# Create a DataFrame with the fetched data and column names
rul_data = pd.DataFrame(rul_data, columns=column_names)

In [9]:
# Display the DataFrame
rul_data = rul_data.drop('index', axis=1)

In [10]:
print(type(rul_data))

<class 'pandas.core.frame.DataFrame'>


# Reading the Turbo Table

In [11]:
turbo_query = sql.SQL("SELECT * FROM turbotable;")

# Execute the query
cur = conn.cursor()
cur.execute(turbo_query)

# Fetch all rows
turbo_data = cur.fetchall()

# Fetch column names from the cursor description
column_names = [desc[0] for desc in cur.description]

# Create a DataFrame with the fetched data and column names
turbo = pd.DataFrame(turbo_data, columns=column_names)

In [12]:
print(type(turbo))

<class 'pandas.core.frame.DataFrame'>


In [13]:
turbo.head()

,index,engine,cycles,altitude (alt),mach_no (mach),throttle_angle (TRA),fan_inlet_temp (T2),LPC_outlet_temp (T24),HPC_outlet_temp (T30),LPT_outlet_temp (T50),...,corrected_fan_speed (NRf),corrected_core_speed (NRc),bypass_ratio (BPR),burner_fuel_air_ratio (farB),bleed_enthalpy (htBleed),demanded_fan_speed (Nf_dmd),demanded_corrected_fan_speed (PCNfR_dmd),HPT_coolant_bleed (W31),LPT_coolant_bleed (W32),source
0,1,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,...,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071,0
1,5,1,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,...,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286,0
2,6,1,6,25.0045,0.6205,60.0,462.54,537.02,1266.38,1048.72,...,2028.01,7868.87,10.8912,0.02,306,1915,84.93,14.28,8.5590,0
3,8,1,8,20.0020,0.7002,100.0,491.19,607.44,1481.69,1252.36,...,2387.99,8049.26,9.2369,0.02,365,2324,100.00,24.33,14.7989,0
4,13,1,13,20.0003,0.7000,100.0,491.19,607.67,1488.74,1256.10,...,2388.04,8048.52,9.1968,0.02,365,2324,100.00,24.40,14.7202,0


In [14]:
turbo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87750 entries, 0 to 87749
Data columns (total 28 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   index                                     87750 non-null  int64  
 1   engine                                    87750 non-null  int64  
 2   cycles                                    87750 non-null  int64  
 3   altitude (alt)                            87750 non-null  float64
 4   mach_no (mach)                            87750 non-null  float64
 5   throttle_angle (TRA)                      87750 non-null  float64
 6   fan_inlet_temp (T2)                       87750 non-null  float64
 7   LPC_outlet_temp (T24)                     87750 non-null  float64
 8   HPC_outlet_temp (T30)                     87750 non-null  float64
 9   LPT_outlet_temp (T50)                     87750 non-null  float64
 10  fan_inlet_pressure (P2)           

In [15]:
turbo.describe().T

,count,mean,std,min,25%,50%,75%,max
index,87750.0,43875.500000,25331.387398,1.0000,21938.25000,43875.50000,65812.7500,87750.0000
engine,87750.0,129.706051,74.360146,1.0000,66.00000,129.00000,192.0000,260.0000
cycles,87750.0,98.335305,66.788208,1.0000,43.00000,90.00000,143.0000,378.0000
altitude (alt),87750.0,23.989409,14.759538,0.0000,10.00460,25.00130,41.9980,42.0080
mach_no (mach),87750.0,0.571707,0.310395,0.0000,0.25060,0.70000,0.8400,0.8420
throttle_angle (TRA),87750.0,94.054017,14.229790,60.0000,100.00000,100.00000,100.0000,100.0000
fan_inlet_temp (T2),87750.0,472.917893,26.414747,445.0000,445.00000,462.54000,491.1900,518.6700
LPC_outlet_temp (T24),87750.0,579.641141,37.308401,535.4100,549.54000,555.92000,607.3100,644.5200
HPC_outlet_temp (T30),87750.0,1419.425250,105.960678,1243.7300,1352.31000,1368.45000,1498.9000,1612.8800
LPT_outlet_temp (T50),87750.0,1204.529599,119.095741,1023.7700,1123.17000,1137.14500,1306.2500,1439.2300


In [16]:
turbo.shape

(87750, 28)

In [17]:
# Split the dataset based on the value of the source column
train_data = turbo[turbo['source'] == 0]
test_data = turbo[turbo['source'] == 1]

In [18]:
print(type(train_data))
print(type(test_data))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [19]:
train_data.head()

,index,engine,cycles,altitude (alt),mach_no (mach),throttle_angle (TRA),fan_inlet_temp (T2),LPC_outlet_temp (T24),HPC_outlet_temp (T30),LPT_outlet_temp (T50),...,corrected_fan_speed (NRf),corrected_core_speed (NRc),bypass_ratio (BPR),burner_fuel_air_ratio (farB),bleed_enthalpy (htBleed),demanded_fan_speed (Nf_dmd),demanded_corrected_fan_speed (PCNfR_dmd),HPT_coolant_bleed (W31),LPT_coolant_bleed (W32),source
0,1,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,...,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071,0
1,5,1,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,...,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286,0
2,6,1,6,25.0045,0.6205,60.0,462.54,537.02,1266.38,1048.72,...,2028.01,7868.87,10.8912,0.02,306,1915,84.93,14.28,8.5590,0
3,8,1,8,20.0020,0.7002,100.0,491.19,607.44,1481.69,1252.36,...,2387.99,8049.26,9.2369,0.02,365,2324,100.00,24.33,14.7989,0
4,13,1,13,20.0003,0.7000,100.0,491.19,607.67,1488.74,1256.10,...,2388.04,8048.52,9.1968,0.02,365,2324,100.00,24.40,14.7202,0


In [20]:
train_data.columns

Index(['index', 'engine', 'cycles', 'altitude (alt)', 'mach_no (mach)',
       'throttle_angle (TRA)', 'fan_inlet_temp (T2)', 'LPC_outlet_temp (T24)',
       'HPC_outlet_temp (T30)', 'LPT_outlet_temp (T50)',
       'fan_inlet_pressure (P2)', 'bypass_duct_pressure (P15)',
       'HPC_outlet_pressure (P30)', 'fan_speed (Nf)', 'core_speed (Nc)',
       'engine_pressure_ratio (epr)', 'HPC_outlet_static_pressure (Ps30)',
       'fuel_ps30_ratio (phi)', 'corrected_fan_speed (NRf)',
       'corrected_core_speed (NRc)', 'bypass_ratio (BPR)',
       'burner_fuel_air_ratio (farB)', 'bleed_enthalpy (htBleed)',
       'demanded_fan_speed (Nf_dmd)',
       'demanded_corrected_fan_speed (PCNfR_dmd)', 'HPT_coolant_bleed (W31)',
       'LPT_coolant_bleed (W32)', 'source'],
      dtype='object')

In [21]:
train_data.shape

(53759, 28)

In [22]:
test_data.head()

,index,engine,cycles,altitude (alt),mach_no (mach),throttle_angle (TRA),fan_inlet_temp (T2),LPC_outlet_temp (T24),HPC_outlet_temp (T30),LPT_outlet_temp (T50),...,corrected_fan_speed (NRf),corrected_core_speed (NRc),bypass_ratio (BPR),burner_fuel_air_ratio (farB),bleed_enthalpy (htBleed),demanded_fan_speed (Nf_dmd),demanded_corrected_fan_speed (PCNfR_dmd),HPT_coolant_bleed (W31),LPT_coolant_bleed (W32),source
53758,53763,1,4,42.0066,0.8410,100.0,445.00,550.17,1349.69,1127.89,...,2387.72,8066.90,9.3961,0.02,332,2212,100.0,10.35,6.4181,1
53760,53760,1,1,9.9987,0.2502,100.0,489.05,605.03,1497.17,1304.99,...,2388.18,8114.10,8.6476,0.03,369,2319,100.0,28.42,17.1551,1
53761,53761,1,2,20.0026,0.7000,100.0,491.19,607.82,1481.20,1246.11,...,2388.12,8053.06,9.2405,0.02,364,2324,100.0,24.29,14.8039,1
53762,53762,1,3,35.0045,0.8400,100.0,449.44,556.00,1359.08,1128.36,...,2387.75,8053.04,9.3472,0.02,333,2223,100.0,14.98,8.9125,1
53763,53769,1,10,20.0001,0.7000,100.0,491.19,607.33,1483.66,1261.66,...,2388.03,8045.67,9.2522,0.02,365,2324,100.0,24.47,14.6186,1


In [23]:
test_data.shape

(33991, 28)

In [24]:
test_data['engine'].nunique()

259

In [25]:
train_data = train_data.drop(['index', 'source'], axis=1)

In [26]:
test_data = test_data.drop(['index', 'source'], axis=1)

# ML

In [27]:
from sklearn.preprocessing import StandardScaler
from scipy import stats
import numpy as np

In [28]:
# Calculate RUL for training data
train_data['RUL'] = train_data.groupby('engine')['cycles'].transform(max) - train_data['cycles']

In [29]:
# Separate features and target variable
X = train_data.iloc[:, 2:-1]
y = train_data['RUL']

In [30]:
X.columns

Index(['altitude (alt)', 'mach_no (mach)', 'throttle_angle (TRA)',
       'fan_inlet_temp (T2)', 'LPC_outlet_temp (T24)', 'HPC_outlet_temp (T30)',
       'LPT_outlet_temp (T50)', 'fan_inlet_pressure (P2)',
       'bypass_duct_pressure (P15)', 'HPC_outlet_pressure (P30)',
       'fan_speed (Nf)', 'core_speed (Nc)', 'engine_pressure_ratio (epr)',
       'HPC_outlet_static_pressure (Ps30)', 'fuel_ps30_ratio (phi)',
       'corrected_fan_speed (NRf)', 'corrected_core_speed (NRc)',
       'bypass_ratio (BPR)', 'burner_fuel_air_ratio (farB)',
       'bleed_enthalpy (htBleed)', 'demanded_fan_speed (Nf_dmd)',
       'demanded_corrected_fan_speed (PCNfR_dmd)', 'HPT_coolant_bleed (W31)',
       'LPT_coolant_bleed (W32)'],
      dtype='object')

# BFE

In [31]:
import statsmodels.api as sm

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

while True:
    max_pvalue = max(model.pvalues)
    if max_pvalue > 0.05:
        feature_to_remove = model.pvalues.idxmax()
        X = X.drop(columns=[feature_to_remove])
        model = sm.OLS(y, X).fit()
    else:
        break

selected_features = X.columns[1:]
print(selected_features)


Index(['altitude (alt)', 'throttle_angle (TRA)', 'fan_inlet_temp (T2)',
       'LPC_outlet_temp (T24)', 'HPC_outlet_temp (T30)',
       'LPT_outlet_temp (T50)', 'fan_inlet_pressure (P2)',
       'bypass_duct_pressure (P15)', 'HPC_outlet_pressure (P30)',
       'core_speed (Nc)', 'engine_pressure_ratio (epr)',
       'HPC_outlet_static_pressure (Ps30)', 'fuel_ps30_ratio (phi)',
       'corrected_core_speed (NRc)', 'bypass_ratio (BPR)',
       'bleed_enthalpy (htBleed)', 'demanded_fan_speed (Nf_dmd)',
       'demanded_corrected_fan_speed (PCNfR_dmd)', 'HPT_coolant_bleed (W31)',
       'LPT_coolant_bleed (W32)'],
      dtype='object')


In [32]:
selected_features.shape

(20,)

# XGB

In [33]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Select only the features identified by backward elimination
X_train_selected = train_data[selected_features.drop('const', errors='ignore')]

# Apply scaling to the selected features
scaler = StandardScaler()
X_train_selected_scaled = scaler.fit_transform(X_train_selected)

# Update test data with selected features and apply the same scaling
X_test_selected = test_data[selected_features.drop('const', errors='ignore')]
X_test_selected_scaled = scaler.transform(X_test_selected)

# Train the XGBoost model
model_xgb = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=5, learning_rate=0.1)
model_xgb.fit(X_train_selected_scaled, y)

# Extract the last cycle for each engine in the test set
last_cycle_indices_xgb = test_data.groupby('engine')['cycles'].idxmax()
X_test_last_cycles_xgb = X_test_selected.loc[last_cycle_indices_xgb]

# Make predictions for the last cycles of each engine
y_pred_last_cycles_xgb = model_xgb.predict(scaler.transform(X_test_last_cycles_xgb))

# True RUL values from the provided RUL file
true_rul_xgb = rul_data.values.flatten()

In [34]:
X_test_selected.columns

Index(['altitude (alt)', 'throttle_angle (TRA)', 'fan_inlet_temp (T2)',
       'LPC_outlet_temp (T24)', 'HPC_outlet_temp (T30)',
       'LPT_outlet_temp (T50)', 'fan_inlet_pressure (P2)',
       'bypass_duct_pressure (P15)', 'HPC_outlet_pressure (P30)',
       'core_speed (Nc)', 'engine_pressure_ratio (epr)',
       'HPC_outlet_static_pressure (Ps30)', 'fuel_ps30_ratio (phi)',
       'corrected_core_speed (NRc)', 'bypass_ratio (BPR)',
       'bleed_enthalpy (htBleed)', 'demanded_fan_speed (Nf_dmd)',
       'demanded_corrected_fan_speed (PCNfR_dmd)', 'HPT_coolant_bleed (W31)',
       'LPT_coolant_bleed (W32)'],
      dtype='object')

In [35]:
# Calculate RMSE, MAE, and R² Score
mse_xgb = mean_squared_error(true_rul_xgb, y_pred_last_cycles_xgb)
rmse_xgb = np.sqrt(mse_xgb)
r2_xgb = r2_score(true_rul_xgb, y_pred_last_cycles_xgb)
mae_xgb = mean_absolute_error(true_rul_xgb, y_pred_last_cycles_xgb)

print(f'XGBoost - Root Mean Squared Error (RMSE): {rmse_xgb}')
print(f'XGBoost - R² Score: {r2_xgb}')
print(f'XGBoost - Mean Absolute Error (MAE): {mae_xgb}')

XGBoost - Root Mean Squared Error (RMSE): 30.344424112331478
XGBoost - R² Score: 0.6816274039371257
XGBoost - Mean Absolute Error (MAE): 22.604580348983234


In [36]:
# pickling the model
import pickle
pickle_out = open("xgb_model.pkl", "wb")
pickle.dump(model_xgb, pickle_out)
pickle_out.close()

In [37]:
# pickling the model
import pickle
pickle_out = open("scaler.pkl", "wb")
pickle.dump(scaler, pickle_out)
pickle_out.close()

In [38]:
train_data[selected_features].head().T

,0,1,2,3,4
altitude (alt),34.9983,25.0005,25.0045,20.0020,20.0003
throttle_angle (TRA),100.0000,60.0000,60.0000,100.0000,100.0000
fan_inlet_temp (T2),449.4400,462.5400,462.5400,491.1900,491.1900
LPC_outlet_temp (T24),555.3200,537.0700,537.0200,607.4400,607.6700
HPC_outlet_temp (T30),1358.6100,1257.7100,1266.3800,1481.6900,1488.7400
LPT_outlet_temp (T50),1137.2300,1047.9300,1048.7200,1252.3600,1256.1000
fan_inlet_pressure (P2),5.4800,7.0500,7.0500,9.3500,9.3500
bypass_duct_pressure (P15),8.0000,9.0300,9.0300,13.6500,13.6500
HPC_outlet_pressure (P30),194.6400,175.0500,175.1700,334.4100,335.3300
core_speed (Nc),8341.9100,7993.2300,7996.1000,8709.1200,8710.3900


In [39]:
y.head()

0    148
1    144
2    143
3    141
4    136
Name: RUL, dtype: int64

In [40]:
train_data[selected_features].tail().T

,53754,53755,53756,53757,53759
altitude (alt),10.0008,0.0015,10.0022,25.0041,35.0036
throttle_angle (TRA),100.0000,100.0000,100.0000,60.0000,100.0000
fan_inlet_temp (T2),489.0500,518.6700,489.0500,462.5400,449.4400
LPC_outlet_temp (T24),605.3500,644.1500,605.8100,537.4800,556.6400
HPC_outlet_temp (T30),1519.8900,1604.8100,1514.3200,1276.2400,1374.6100
LPT_outlet_temp (T50),1329.3700,1427.5300,1324.1200,1057.9200,1145.5200
fan_inlet_pressure (P2),10.5200,14.6200,10.5200,7.0500,5.4800
bypass_duct_pressure (P15),15.5000,21.6100,15.5000,9.0400,8.0100
HPC_outlet_pressure (P30),392.5500,552.0200,392.4600,175.1100,194.3300
core_speed (Nc),8902.5200,9184.2300,8915.8600,8114.3300,8478.4400


In [41]:
y.tail(5)

53754    6
53755    5
53756    3
53757    2
53759    0
Name: RUL, dtype: int64